# 1. Data: Wikipedia
WE will use the <a href="https://pypi.org/project/wikipedia/">wikipedia python api</a> to download some text data. This api allows you to download wikipedia pages and create your text corpus programmatically.
**Study the API docs. You can also do this lab in a different language than English.**

The procedure we will use:
1. Download texts in two different categories. For example, if one of your categories is 'political figures' then one of your searches could be 'Hilary Clinton' and then we can download all the texts in wikipedia that contain the terms 'Hilary Clinton' in the title. Try to have around ten possible names in each category.
2. Split the texts into sentences and each sentence into tokens, representing it as a list (using the nltk tools).
3. It will be convenient to add a STARTSENT and ENDSENT marker to each one.

**YOu need to complete the code below (preferably by generalising the for loop to populate both corpora)**

In [ ]:
import wikipedia
from nltk import word_tokenize, sent_tokenize

#Feel free to change these, but choose potential titles that belong to the same category
#List of topics or names in category 1, e.g. rock musicians
cat1 = ['Jimi Hendrix', 'Janis Joplin', 'Steve Winwood', 'Siouxsie Sioux'] 
cat2 = [] #List of wiki articles in category 2 -- enter your own

corp1 = [] #This will be a list of sentences in texts in cat1, each represented as a list
corp2 = [] #Do the same for this one in cat 2

for c in cat1:    
    page_hits = wikipedia.search(c) #all the pages with this search term in the title
    #print(page_hits) #Uncomment if you want to see the hits
    
    for hit in page_hits:
        page = wikipedia.page(hit) #Download the actual page
        text = page.content #the actual text
        
        #tokenise and add to corpus - NB This is a list of lists
        for s in sent_tokenize(text): 
            corp1.append(['STARTSENT'] + word_tokenize(s) + ['ENDSENT'])


## 1.1 Train, dev and test
We're going to need training and test data, but also **dev** data, which we'll use below for determining the best smoothing parameter.

Complete the function below which, given your dataset and proportions `tr`, `dv` and `ts` (for train, dev and test respectively -- e.g. 0.8, 0.1, and 0.1) returns a random split of your data into three different sets. (*Hint*: Look at the similar function we used in our Naive Bayes lab. But remember this time the data is just a python list). Create separate train/dev/test splits for both your corpora.



In [ ]:
import numpy as np

def split_train_test(data, tr, dv, ts):
    '''Splits data into train, dev and test sets based on given proportions.'''
    
    shuffled_indices = np.random.permutation(len(data)) #randomise indices
    tr_size = int(len(data) * tr)
    dv_size = int(len(data) * dv)
    ts_size = int(len(data) * ts)
    
    #indices of items in train, test and dev
    train_idx = shuffled_indices[:tr_size] #training set indices
    dev_idx = shuffled_indices[tr_size: tr_size+dv_size]
    test_idx = shuffled_indices[tr_size+dv_size:]
    
    train_set = [data[i] for i in train_idx]
    dev_set = [data[i] for i in dev_idx]
    test_set = [data[i] for i in test_idx]
    return train_set, dev_set, test_set

corp1_train, corp1_dev, corp1_test = split_train_test(corp1, 0.8, 0.1, 0.1)


# 2. Models

We'll use some biult-in nltk functions to build our models. NLTK provides the functions `bigram`, `trigram` to extract n-grams of sizes 2 and 3. In addition, we'll use a python `defaultdict` to hold our ngram models. This is available in python's `collections` library. Though basically a python dictionary, it provides some high-performance functionality.

The strategy we'll use is exemplified below:

1. Extract n-grams of different sizes and build a python default dict mapping histories to the final word in the ngram, to frequencies. (e.g. in a trigram model, we have (word1, word2): count)
2. Convert the frequencies to probabilities using add-k smoothing, for some variable quantity `k`

An example is done for you below, for bigrams. **You need to generalise this function to create n-grams of variable lengths (from 1 to 3).** 

The additional function `to_probabilities` should work for any order of ngram. **You also need to complete this function.** Note that this function takes a parameter `k`, which is 1 by default. The idea is to use `k=1` for add-one smoothing, or indeed any value of `k` (if 0, it's just a Maximum Likelihood Estimate).
 
Some things to consider:
* To make the function generalisable, we can represent keys in our dictionary as tuples of variable length. For an ngram of length n, the dictionary will have the form `model[(history)][wn] = count`. For example, the trigram *I like sugar* would be represented as `model[('I', 'like')]['sugar'] = count` while the bigram *I like* would be `model[('I')]['like'] = count`.
* The example below does not lowercase words. This means that 'Jimi' and 'jimi' are not the same word. Consider whether you want this or not.
* For a given set of ngrams starting with some history `h`, probabilities should sum to approximately 1. Check that this is the case for a few examples. (*Hint*: in a bigram model, `model[(h)].values()` gives you all the frequencies for any bigram starting with history `h`)

In [ ]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

#Builds bigrams - change this to build ngrams of various lengths up to 3
def build_bigrams(data):
    #the incantation below creates a dictionary of dictionaries
    #{word: {word: frequency}}
    #for trigrams w1,w2,w3, we can do: model[(w1,w2)][w3] so keys are tuples of length 2
    model = defaultdict(lambda: defaultdict(lambda: 0))
    
    for s in data:                
        for w1, w2 in bigrams(s):           
            model[tuple(w1)][w2] += 1 
    return model

#Convert counts to probabilities
# Add k for smoothing (default k = 1)
def to_probabilities(model, k=1):  
    vocab_size = len(model)
    prob_model = defaultdict(lambda: defaultdict(lambda: 0))
                  
    #transform to probabilities
    # with smoothing parameter
    for history in model:
        total_count = float(sum(model[history].values()))
        
        for word in model[history]:
            prob_model[history][word] = (model[history][word] + k) / (total_count+ (vocab_size*k))
    
    return prob_model
    
#Build model with frequencies
c1_bigram_freq = build_bigrams(corp1_train)

#Convert our models to (smoothed) probabilities
c1_bigram_prob = to_probabilities(c1_bigram_freq)          


## 2.1 Working with unseen values

Our model won't contain all ngrams. If we use a smoothing parameter `k`, we can also use `k` to estimate the probability of an unseen case, given the vocabulary size. The function below needs to be completed to achieve this. 

Remember that it is safe to do this because we have discounted all our actual probabilities by an amount proportional to `k`. If we don't do this, unseens are just assigned probability 0.

**Complete the function below to return the probability of a new, previously unseen ngram (assuming we pass the function the ngram in the form of a tuple)**

In [ ]:
#Compute probability of unseen bigrams (w1,w2) with add-k
#NB: Pass the model with counts, not probabilities!
def unseen_prob(freq_model, k, ngram_tuple):
    if k == 0:
        return 0
    
    voc_size = len(freq_model)
    
    #unigram case 
    if len(ngram_tuple) == 1:
        total_freq = sum(freq_model.values())
        return k / (total_freq + voc_size)

    #Ngrams for n > 1
    else:
        history = ngram_tuple[:-1]
        hist_freq = sum(freq_model[history].values())
        return k / (hist_freq + voc_size)
    

# 3. Perplexity

Given a model, we evaluate by computing its perplexity on a test set.

Complete the function below (your code should go where indicated). It should take an ngram model (both frequencies and probabilities) and a set of ngrams (we can use our function above to get them) extracted from a test set or dev set, and return the perplexity value.

Things to consider:
1. If you used some value of `k` for smoothing your training ngram model, you need that `k` for perplexity. This is because, if there's some ngram in the test set which happens not to be in training (and there will be), you use `k` as your estimate. If you left `k` equal to 0, the perplexity calculation must simply ignore any ngrams in dev/test that are not in train. This is undesirable (so do train your model with some value of `k`).
2. YOu don't need to have probabilities for the test or dev set, just the ngrams themselves (we'll use our function above to obtain them).
3. Don't multiply probabilities as you're likely to get overflow errors. Add logs instead. WE typically use logs to base 2


In [1]:
#Perplexity function: given a trained model and a list of ngrams from dev/test
#Also needs smoothing parameter k if used, else zero by default
#test_n is the number of tokens in the test corpus
def perplexity(prob_model, freq_model, test_ngrams, test_n, k=1):    
    probabilities = [] #accumulate individual probs here
    
    #Iterate through the test ngram set
    # Compute the perplexity by extracting the probability for a TEST ngram
    # using the TRAINED_MODEL
    for history in test_ngrams:
        if history in model:
            for last_word in test_ngrams[history]:
                if last_word in model[history]:
                    probabilities.append(np.log2(model[history][last_word]))

                elif k > 0: #useless to include for k=0
                    unseen_p = unseen_prob(freq_model, k, history + tuple(last_word))
                    probabilities.append(np.log2(unseen_p))
        elif k > 0:
            unseen_p = unseen_prob(freq_model, k, history + tuple(last_word))
            probabilities.append(np.log2(unseen_p))
    
    #take the sum of log probs and multiple by -1/N
    prob_sum = np.sum(probabilities) * -(1/test_n)
    #take exponent to 
    perplexity = np.power( 2, prob_sum )
    return perplexity
        
#Example: get the bigrams for the corp1 dev set and compute perplexity on that
#If we used k =/= 0 to train our model, we need to supply this to our perplexity estimate
# This is used for any ngrams which are not found in the train set
c1_bigrams_dev = build_bigrams(corp1_dev)
dev_size = np.sum([len(s) for s in corp1_dev])
pp_dev = perplexity(c1_bigram_prob, c1_bigram_freq, c1_bigrams_dev, dev_size, k=1)
print(pp_dev)

#Same for test set
c1_bigrams_test = build_bigrams(corp1_test)
test_size = np.sum([len(s) for s in corp1_test])
pp_test = perplexity(c1_bigram_prob, c1_bigram_freq, c1_bigrams_test, test_size, k=1)
print(pp_test)

NameError: name 'build_bigrams' is not defined

## 3.1 Optimising k

At this point, you have a function to build language models, and a function to compute their perplexity. Now, let's use perplexity to find the best amount `k` to add during `add-k` smoothing.

Do the following:

1. Train different langauge models (bigram or trigram) with values of k ranging from `0.1` to `1` at increments of `0.1`. **NB** To train a trigram model, you'll need to reconsider the function `build_bigrams` above. How can you change it to make it build trigram models?
2. For each model, compute its perplexity **on the dev set**
3. Choose the model with the lowest perplexity and compute its final perplexity on the test set.


## 3.2 The effect of data

Having established your optimal value for smoothing, try computing the perplexity of a model trained on your first corpus (corp1), on the test set of a second corpus (corp2).

You would typically observe an increase in perplexity here: your model will do better on a test set sampled from the same data, compared to data related to a different topic.

Consider why this could be. What does this tell you about how models capturre aspects of content and style, from one type of text (or topic) to another?

# 4. Random generation

Given an n-gram model, where start/end symbols have also been included (as they were above), we can write a random sentence generator. With a bigram model, for example, this would proceed as follows:

1. Choose an ngram that begins with `STARTSENT`, `(STARTSENT, x)` with some probability proportional to the model probability. Add `x` to your sentence.
2. Until the `ENDSENT` symbol is reached: take the last word of the previously sampled ngram `(_, x)`, and find another ngram `(x, y)` that begins with `x`. Concatenate `y` to the sentence.
3. Stop if you sample an ngram whose last word is `ENDSENT`